In [4]:
import pandas as pd
import numpy as np
from symbol import Symbol
from index import Index
from portifolio import Portifolio
from datetime import datetime
import os
import matplotlib.pyplot as plt
from bqplot import pyplot as bplt
import seaborn as sns
from dateutil.relativedelta import relativedelta
from investing import *

sns.set(context='notebook', style="dark", palette="muted", color_codes=True)

%matplotlib inline

In [5]:
s= Symbol()
i = Index()
p = Portifolio()

In [6]:
s.symbol_meta.head()

,name_of_company,isin_number,date_of_listing,sector,mcap,pe_ratio,div_yield,promoter_holding,fii_holding,dii_mf_holding,others_holding
symbol,,,,,,,,,,,
20microns,20 Microns Limited,INE144J01027,2008-10-06,metals,142.0,13.690,0.00,50.0,0.0,0.0,50.0
3iinfotech,3i Infotech Limited,INE748C01020,2005-04-22,it,623.0,-0.544,0.00,3.0,4.0,34.0,59.0
3mindia,3M India Limited,INE470A01017,2004-08-13,trading,13855.0,65.500,0.00,75.0,9.0,3.0,13.0
63moons,63 moons technologies limited,INE111B01023,2005-06-20,Information Technology,374.0,NaN,6.16,46.0,11.0,0.0,43.0
8kmiles,8K Miles Software Services Limited,INE650K01021,2014-01-29,it,1942.0,23.590,0.00,63.0,0.0,2.0,34.0


In [12]:
investing_info = get_investing_company_info()
investing_info.market_cap/10 ** 7

symbol
20microns       142.000
3iinfotech      665.000
3mindia       14054.000
63moons         382.000
8kmiles        1888.000
a2zinfra        559.000
aartidrugs     1478.000
aartiind       5924.000
aarveeden       211.000
aban           1476.000
abb           23536.000
abbotindia     9415.000
abfrl         11438.000
abirlanuvo    17832.000
acc           26866.000
accelya        2297.000
ace             548.000
adanient      10184.000
adaniports    62947.000
adanipower    12966.000
adanitrans     7330.000
adffoods        333.000
adhunik          93.242
adhunikind      498.000
adlabs          613.000
adorweld        422.000
advanihotr      278.000
advenzymes     4024.000
aegischem      4504.000
afl             178.000
                ...    
welinv              NaN
welspunind     7691.000
wendt           374.000
wheels         1503.000
whirlpool     12458.000
willamagor       62.342
windmachin      298.000
winsome          12.727
wipro         15032.000
wockpharma     7390.000
wonderla 

In [5]:
ret = s.get_symbol_data()
close = s.get_symbol_data('close')
volume = s.get_symbol_data('volume')
ret = ret['2016']
close = close['2016']
volume = volume['2016']
iret = i.get_index_returns('nifty_50')
iclose = i.get_index_close(index_list='nifty_50')
iret = iret['2016']
iclose = iclose['2016']

In [6]:
from helpers import num_missing
null_count = ret.apply(num_missing, start='beginning')
symbol_list = null_count[null_count == 0].index
symbol_list = list(symbol_list)

In [7]:
ret = ret[symbol_list]
close = close[symbol_list]
volume = volume[symbol_list]
adj_close = pd.DataFrame(p.prices(ret))
adj_close = (close/close.ix[-1, :])*close.ix[-1].round(4)

In [8]:
def high_low(adj_close):
    des = pd.DataFrame(index=adj_close.columns)
    des['low'] = adj_close.min()
    des['high'] = adj_close.max()
    des['mean'] = adj_close.max()
    des['median'] = adj_close.median()
    
    return des
close_stats = high_low(adj_close)
volume_stats = high_low(volume).astype(int)
close_stats.head()

,low,high,mean,median
symbol,,,,
20microns,25.45,43.15,43.15,32.25
3iinfotech,3.80,6.80,6.80,5.05
3mindia,9521.50,14939.55,14939.55,12638.25
63moons,69.55,125.45,125.45,81.05
8kmiles,591.30,2483.70,2483.70,1868.55


In [9]:
def returns_stats(ret, iret):
    iret = iret.nifty_50
    des = pd.DataFrame(index=ret.columns)
    des['sum_returns'] = ret.sum()
    des['min_returns'] = ret.min()
    des['high_returns'] = ret.max()
    des['mean_returns'] = ret.mean()
    des['median_returns'] = ret.median()
    
    risk_free_rate, _ = s.set_risk_free_rate(ret)
    des['hpr'] = np.exp(ret.sum())*100 - 100
    des['hpr_rf'] = np.exp(ret.sum() - risk_free_rate.sum())*100 - 100
    des['hpr_bench'] = np.exp(ret.sum() - iret.sum())*100 - 100

    des['std_dev'] = ret.std()
    des['lpm'] = p.lower_partial_moment(ret)
    des['hpm'] = p.higher_partial_moments(ret)
    des['pos_pctile'] = ret[ret>0].count() / ret.count()
    
    
    
    return des
ret_stats = returns_stats(ret, iret)
ret_stats.head()

,sum_returns,min_returns,high_returns,mean_returns,median_returns,hpr,hpr_rf,hpr_bench,std_dev,lpm,hpm,pos_pctile
symbol,,,,,,,,,,,,
20microns,-0.102321,-0.098218,0.178966,-0.000414,0.000000,-9.726027,-15.575530,-12.366710,0.029477,0.019607,0.021934,0.465587
3iinfotech,0.152677,-0.106295,0.163629,0.000618,0.000000,16.494845,8.946303,13.087153,0.034119,0.021215,0.026641,0.368421
3mindia,-0.024374,-0.082843,0.153247,-0.000099,-0.000478,-2.407974,-8.731666,-5.262723,0.020390,0.012013,0.016425,0.469636
63moons,-0.481592,-0.085544,0.128607,-0.001950,-0.002105,-38.220120,-42.223285,-40.027298,0.026102,0.017589,0.019314,0.449393
8kmiles,-0.287392,-0.214198,0.174936,-0.001164,-0.003109,-24.978228,-29.839430,-27.172755,0.036794,0.025412,0.026531,0.408907


In [10]:
capm_prices = p.calculate_capm(close, iclose).set_index('symbol').drop(['benchmark'], axis=1)
capm_prices.head()

,alpha,beta,std_dev,r_square
symbol,,,,
20microns,-15.519320,0.005942,4.458844,0.450033
3iinfotech,-2.836240,0.000970,0.735341,0.440818
3mindia,-3019.230725,1.874128,1294.924976,0.530793
63moons,128.048631,-0.005548,9.888959,0.079753
8kmiles,3249.604852,-0.198135,542.858917,0.033757


In [11]:
def returns_ratios(returns, iret):
    returns_describe = p.calculate_capm(returns, iret).set_index('symbol').drop(['benchmark'], axis=1)
    sharpe_ratio = p.sharpe_ratio(returns)
    returns_describe = returns_describe.join(sharpe_ratio)
    sortino_ratio = p.sortino_ratio(returns, 0)
    returns_describe = returns_describe.join(sortino_ratio)
    upside_potential_ratio = p.upside_potential_ratio(returns, 0)
    returns_describe = returns_describe.join(upside_potential_ratio)
    return returns_describe
ratios_returns = returns_ratios(ret, iret)
ratios_returns.head()

,alpha,beta,std_dev,r_square,sharpe_ratio,sortino_ratio,upside_potential_ratio
symbol,,,,,,,
20microns,-0.000519,1.104112,0.029478,0.127880,-0.366208,-0.550556,1.118674
3iinfotech,0.000506,1.053774,0.034120,0.086944,0.160113,0.257514,1.255779
3mindia,-0.000260,0.729387,0.020389,0.116645,-0.285700,-0.484908,1.367219
63moons,-0.002068,1.012637,0.026104,0.137171,-1.339892,-1.988531,1.098048
8kmiles,-0.001161,1.813710,0.036794,0.221481,-0.614077,-0.889115,1.044015


In [12]:
def min_max_correlation(returns):
    corr_mat = pd.DataFrame(index=returns.columns)
    correlation = returns.corr()
    correlation = correlation.replace(1, np.nan)
    corr_mat['min_corr'] = correlation.min()
    corr_mat['min_corr_symbol'] = correlation.idxmin()
    corr_mat['max_c0rr'] = correlation.max()
    corr_mat['max_corr_symbol'] = correlation.idxmax()
    return corr_mat
min_max_corr = min_max_correlation(ret)
min_max_corr.head()

,min_corr,min_corr_symbol,max_c0rr,max_corr_symbol
symbol,,,,
20microns,0.017390,cyient,0.482799,bbtc
3iinfotech,-0.010682,kirlosind,0.465779,jswenergy
3mindia,-0.039266,tokyoplast,0.475518,insecticid
63moons,-0.037567,swanenergy,0.480993,il&fstrans
8kmiles,0.003430,mangtimber,0.496592,prismcem


In [13]:
from market import INDEX_META
INDEX_META.index
smsa = pd.DataFrame(index=s.symbol_list.index.copy())
for index_type in INDEX_META.category.unique():
    data = s.get_symbol_index(index_type)
    for index in data.columns:
        data[index] = data[index].replace(True, index).replace(False, np.nan)
    data = data.apply(lambda x: list(x.dropna().values), axis=1).rename(index_type)
    smsa = smsa.join(data)
smsa.head()

,mkt_index,stg_index,sec_index,thematic_index
symbol,,,,
20microns,[],[],[],[]
3iinfotech,[],[],[],[]
3mindia,"[nifty_midcap_150, nifty_free_float_midsmallca...",[],[],[]
63moons,[],[],[],[]
8kmiles,"[nifty_smallcap_250, nifty_free_float_midsmall...",[],[],[]


In [106]:
ret = s.get_symbol_data(volume=10000, start='2016-12', mcap=1000)
ret = ret.fillna(0)
min_max_corr = min_max_correlation(ret)

In [108]:
p.beta(ret.motilalofs, ret.edelweiss)

,edelweiss
motilalofs,0.422682


In [102]:
new = returns_ratios(ret, iret)

In [104]:
new.sort_values('upside_potential_ratio')

,alpha,beta,std_dev,r_square,sharpe_ratio,sortino_ratio,upside_potential_ratio
symbol,,,,,,,
divislab,-0.008791,1.408796,0.033828,0.116321,-4.568405,-5.329405,0.406953
crompgreav,-0.012392,-0.201087,0.029747,0.003065,-4.416547,-5.222182,0.445808
canbk,-0.008290,1.435634,0.017088,0.473369,-4.621945,-5.053336,0.567145
sunpharma,-0.005151,1.120840,0.020794,0.194852,-2.758491,-3.563072,0.599812
jklakshmi,-0.007286,0.884550,0.020856,0.120640,-3.896850,-4.843152,0.601412
marksans,-0.004784,0.709081,0.011241,0.266878,-3.827546,-4.648215,0.614424
shriramepc,-0.005816,1.292203,0.017230,0.377209,-5.014101,-6.334124,0.622056
rcom,-0.004339,1.291553,0.026081,0.164475,-3.158819,-3.945561,0.638506
bfutilitie,-0.005956,0.952128,0.017489,0.198770,-4.585870,-5.568306,0.660769
